In [1]:
import os
import re
import pandas as pd

### Đọc dữ liệu và lưu vào file để tạo file nguyên bản đầu tiên

In [2]:
folder_path = r"Fixations"  # m đọc và thay tên folder gốc chứa data của mình vào đây
output_csv = "original_file.csv" # tên file sau khi được ghi ra
expected_cols = ["IMAGE","FIX_INDEX","FIX_DURATION","FIX_X","FIX_Y","FIX_PUPIL"]


# hàm đổi tên đọc trong cột source file và đổi sang int để sau đó lưu sang một cột khác
def extract_numeric_from_filename(fname):
    """
    Lấy chuỗi số đầu tiên trong tên file, ví dụ "000.xlsx" -> 0, "201_extra.xlsx" -> 201.
    Trả về int hoặc None nếu không tìm thấy số.
    """
    m = re.search(r"(\d+)", fname)
    return int(m.group(1)) if m else None
files = [f for f in os.listdir(folder_path) if f.lower().endswith((".xls", ".xlsx"))]
file_infos = []
for f in files:
    num = extract_numeric_from_filename(f)
    if num is None:
        file_infos.append((f, -1))
    else:
        file_infos.append((f, num))


# sắp xếp theo số (tăng dần). Những file không có số sẽ đứng cuối.
file_infos = sorted(file_infos, key=lambda t: t[1])

records = []
bad_files = []      # lưu file bị lỗi / thiếu cột
skipped_files = []  # lưu file bị skip do lỗi đọc

for fname, num in file_infos:
    path = os.path.join(folder_path, fname)
    try:
        df = pd.read_excel(path, engine="openpyxl")
    except Exception as e:
        skipped_files.append((fname, str(e)))
        continue
    cols_lower = [c.strip().upper() for c in df.columns]
    if not all(col in cols_lower for col in expected_cols):
        bad_files.append(fname)
        continue
    col_map = {orig: orig.strip().upper() for orig in df.columns}
    df = df.rename(columns=col_map)
    df = df[expected_cols].copy()
    numeric_id = num if num != -1 else extract_numeric_from_filename(fname)
    # nếu file tên ko có số sẽ để numeric_id = None
    label = 1 if (numeric_id is not None and numeric_id > 200) else 0
    df["source_file"] = fname
    df["numeric_id"] = numeric_id
    df["label"] = label
    df["FIX_INDEX"] = pd.to_numeric(df["FIX_INDEX"], errors="coerce").astype("Int64")
    df["FIX_DURATION"] = pd.to_numeric(df["FIX_DURATION"], errors="coerce")
    df["FIX_X"] = pd.to_numeric(df["FIX_X"], errors="coerce")
    df["FIX_Y"] = pd.to_numeric(df["FIX_Y"], errors="coerce")
    df["FIX_PUPIL"] = pd.to_numeric(df["FIX_PUPIL"], errors="coerce")
    df = df.dropna(subset=["FIX_INDEX","FIX_X","FIX_Y"])
    records.append(df)

# concat tất cả
if records:
    merged_df = pd.concat(records, ignore_index=True)
else:
    merged_df = pd.DataFrame(columns = expected_cols + ["source_file","numeric_id","label"])

# save
merged_df.to_csv(output_csv, index=False)

# summary báo cáo
print(f"Đã đọc {len(records)} file hợp lệ. Tổng số dòng: {len(merged_df)}")
if bad_files:
    print("Các file bị thiếu cột (bị bỏ qua):", bad_files)
if skipped_files:
    print("Các file bị lỗi khi đọc (bị bỏ qua):", skipped_files)

# hiển thị 5 dòng đầu

merged_df.head()

Đã đọc 160 file hợp lệ. Tổng số dòng: 225159


,IMAGE,FIX_INDEX,FIX_DURATION,FIX_X,FIX_Y,FIX_PUPIL,source_file,numeric_id,label
0,outman_054.jpg,1,205,518.5,371.3,1177,000.xlsx,0,0
1,outman_054.jpg,2,90,275.1,282.2,1262,000.xlsx,0,0
2,outman_054.jpg,3,305,600.3,271.1,1265,000.xlsx,0,0
3,outman_054.jpg,4,269,635.2,269.2,1326,000.xlsx,0,0
4,outman_054.jpg,5,318,597.3,266.0,1366,000.xlsx,0,0


In [3]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225159 entries, 0 to 225158
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   IMAGE         225159 non-null  object 
 1   FIX_INDEX     225159 non-null  Int64  
 2   FIX_DURATION  225159 non-null  int64  
 3   FIX_X         225159 non-null  float64
 4   FIX_Y         225159 non-null  float64
 5   FIX_PUPIL     225159 non-null  int64  
 6   source_file   225159 non-null  object 
 7   numeric_id    225159 non-null  int64  
 8   label         225159 non-null  int64  
dtypes: Int64(1), float64(2), int64(4), object(2)
memory usage: 15.7+ MB


In [4]:
merged_df["IMAGE"].value_counts()
# h ta sẽ index các image thành các số int và bước sau sẽ đếm xem mỗi người nhìn bao  nhiêu ảnh và các ảnh có gióng nhau không

IMAGE
mood_39.jpg    2659
mood_43.jpg    2636
mood_38.jpg    2600
mood_44.jpg    2593
mood_41.jpg    2587
               ... 
noi_052.jpg    1967
low_032.jpg    1951
low_031.jpg    1929
noi_051.jpg    1913
low_034.jpg    1895
Name: count, Length: 100, dtype: int64

In [5]:
merged_df["image_id"] = merged_df["IMAGE"].astype("category").cat.codes
'''Sau khi chạy dòng code trên, Pandas sẽ gán số cho các giá trị duy nhất (thường theo thứ tự bảng chữ cái),
nó chuyển đổi dữ liệu dạng chuỗi (text) trong cột "IMAGE" thành các số nguyên duy nhất và lưu vào một cột mới tên là "image_id".
ví dụ: 
        train_01.jpg ->0 ;
        train_02.jpg ->1 ;
        train_03.jpg ->2 ;
'''

'Sau khi chạy dòng code trên, Pandas sẽ gán số cho các giá trị duy nhất (thường theo thứ tự bảng chữ cái),\nnó chuyển đổi dữ liệu dạng chuỗi (text) trong cột "IMAGE" thành các số nguyên duy nhất và lưu vào một cột mới tên là "image_id".\nví dụ: \n        train_01.jpg ->0 ;\n        train_02.jpg ->1 ;\n        train_03.jpg ->2 ;\n'

In [6]:
merged_df.head(20)

,IMAGE,FIX_INDEX,FIX_DURATION,FIX_X,FIX_Y,FIX_PUPIL,source_file,numeric_id,label,image_id
0,outman_054.jpg,1,205,518.5,371.3,1177,000.xlsx,0,0,53
1,outman_054.jpg,2,90,275.1,282.2,1262,000.xlsx,0,0,53
2,outman_054.jpg,3,305,600.3,271.1,1265,000.xlsx,0,0,53
3,outman_054.jpg,4,269,635.2,269.2,1326,000.xlsx,0,0,53
4,outman_054.jpg,5,318,597.3,266.0,1366,000.xlsx,0,0,53
5,outman_054.jpg,6,585,575.4,269.3,1442,000.xlsx,0,0,53
6,outman_054.jpg,7,531,610.9,274.1,1508,000.xlsx,0,0,53
7,outman_054.jpg,8,336,595.0,333.7,1577,000.xlsx,0,0,53
8,outman_054.jpg,9,400,597.8,599.6,1687,000.xlsx,0,0,53
9,outman_054.jpg,10,325,550.1,580.0,1756,000.xlsx,0,0,53


In [7]:
merged_df.to_csv("original_file.csv", index=False)#lưu lại bảng đã chỉnh sửa vào file gốc này


### kiểm tra xem mỗi người nhìn bao nhiêu ảnh

In [8]:
result_df = merged_df.groupby('numeric_id')['image_id'].nunique().reset_index() 

In [9]:
result_df.head(160)# vậy kết luận là tất cả người tham gia kiểm tra đều nhìn đủ 100 ảnh 

,numeric_id,image_id
0,0,100
1,1,100
2,2,100
3,3,100
4,4,100
...,...,...
155,298,100
156,299,100
157,301,100
158,302,100


In [10]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):# hiển thị tất cả số ảnh mà mỗi người nhìn 
    display(result_df)

,numeric_id,image_id
0,0,100
1,1,100
2,2,100
3,3,100
4,4,100
5,5,100
6,6,100
7,7,100
8,8,100
9,9,100
